In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint

/home/hao/anaconda3/envs/MLOntology/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#global variabls

directory_path = "/home/hao/AnacondaProjects/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"


In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)

prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [5]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                childID = get_trailing_number(splitted[1])
                parentID = get_trailing_number(splitted[2].replace("\r\n", ""))
                conceptPairList.append([childID, parentID , 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = data_path + "ontHierarchy_owl_ncit.txt"
read_pair(pair_file)

checkpairs = conceptPairList[10:15]
print(checkpairs)
print(len(conceptPairList))

[['4861', '7318', 1], ['87152', '87150', 1], ['87153', '140032', 1], ['87154', '87153', 1], ['87155', '87153', 1]]
16533


In [6]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                childID = get_trailing_number(splitted[0])
                notparentID = get_trailing_number(splitted[1].replace("\r\n", ""))
                conceptNotPairList.append([childID, notparentID, 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = data_path + "taxNotPairs_owl_ncit.txt"
read_not_pair(notPair_file)


first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

[['7918', '9151', 0], ['7918', '48612', 0], ['7918', '48613', 0], ['7918', '91231', 0], ['7918', '66753', 0]]
32197


In [7]:
# In-place shuffle
random.shuffle(conceptNotPairList)
conceptNotPairList = conceptNotPairList[:len(conceptPairList)]

print(len(conceptNotPairList))

16533


In [8]:
vector_model_file = vector_model_path + "model0"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[(9748, 0.718510627746582),
 (5881, 0.7043671011924744),
 (3393, 0.6982367038726807),
 (7167, 0.6774163246154785),
 (7095, 0.670405387878418),
 (4230, 0.6645699739456177),
 (3973, 0.6582036018371582),
 (9004, 0.6564831137657166),
 (2440, 0.6524202823638916),
 (3529, 0.6482406854629517)]


In [23]:
vector_model.docvecs[7918]

KeyError: "tag '7918' not seen in training corpus/invalid"

In [ ]:
vector_model_file = vector_model_path + "model1"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

In [9]:
feature_number = 1024

def readFromPairList(id_pair_list, id_notPair_list):
    pair_list = id_pair_list + id_notPair_list
    random.shuffle(pair_list)
    idpairs_list =[]
    label_list =[]
    for i, line in enumerate(pair_list):      
        idpairs_list.append([line[0], line[1]])
        label_list.append(line[2])
    return idpairs_list, label_list

idpairs_list, label_list= readFromPairList(conceptPairList, conceptNotPairList)

print(label_list[:20])

[0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(idpairs_list, label_list, test_size = 0.2, shuffle= True)
print(X_train[:20])
print(X_train[:20])
print(y_train[:20])
print(y_test[:20])

[['133442', '4024'], ['115033', '45709'], ['129707', '127155'], ['3540', '3219'], ['8949', '2919'], ['8863', '4341'], ['6149', '39850'], ['27843', '66950'], ['115138', '134515'], ['5280', '3139'], ['7995', '62277'], ['140418', '91202'], ['3876', '3157'], ['6587', '8964'], ['6429', '6421'], ['5512', '7399'], ['8927', '8853'], ['96948', '7124'], ['5636', '5637'], ['39998', '40009']]
[['133442', '4024'], ['115033', '45709'], ['129707', '127155'], ['3540', '3219'], ['8949', '2919'], ['8863', '4341'], ['6149', '39850'], ['27843', '66950'], ['115138', '134515'], ['5280', '3139'], ['7995', '62277'], ['140418', '91202'], ['3876', '3157'], ['6587', '8964'], ['6429', '6421'], ['5512', '7399'], ['8927', '8853'], ['96948', '7124'], ['5636', '5637'], ['39998', '40009']]
[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1]


In [11]:
feature_number = 1024

def getVectorFromModel(concept_id, conceptLabelDict, model):
    if concept_id in model.docvecs:
        concept_vector= model.docvecs[concept_id]
    else:
        print("%s not found, get inferred vector "%(concept_id))
        concept_label = conceptLabelDict[concept_id]
        concept_vector= model.infer_vector(concept_label.split())
    return concept_vector

def getVector(line, conceptLabelDict, model):        
    a = getVectorFromModel(line[0], conceptLabelDict, model)
    b = getVectorFromModel(line[1], conceptLabelDict, model)
    c = np.array((a, b))
    c = c.T 
    c = np.expand_dims(c, axis=2)
    print(c.shape)
    return c

In [12]:
def stackVector(vector):
    from numpy import dstack
    return dstack((vector, vector, vector))

In [13]:
nb_classes=2 

def generator(x_samples, y_samples, train_flag, batch_size=64):
    samples = list(zip(x_samples, y_samples))
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            X_samples = []
            Y_samples= []
            for batch_sample in batch_samples:
                pair_list = batch_sample[0]
                data_vector = getVector(pair_list, conceptLabelDict, vector_model)
                data_vector = stackVector(data_vector)
                X_samples.append(data_vector)
                class_label = batch_sample[1] 
                Y_samples.append(class_label)
                
            X_samples = np.array(X_samples).astype('float32')
            Y_samples = np.eye(nb_classes)[Y_samples]
            print('one batch ready')
            yield shuffle(X_samples, Y_samples)

In [14]:
set_batch_size = 64

# compile and train the model using the generator function
train_generator = generator(X_train, y_train, train_flag=True, batch_size=set_batch_size)
validation_generator = generator(X_test, y_test, train_flag=False, batch_size=set_batch_size)

In [15]:
from keras import applications
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np

In [16]:
model = applications.resnet50.ResNet50(weights=None, include_top = True, classes=2)
print('Model loaded')

for layer in model.layers:
    layer.trainable = True
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Model loaded


In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger(directory_path + 'resnet18_cifar10.csv')

In [19]:
myepochs = 10

model.fit_generator(train_generator, 
                    steps_per_epoch=len(X_train)//set_batch_size, 
                    epochs=myepochs,
                    validation_data=validation_generator,
                    validation_steps=len(X_test)//set_batch_size,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

Epoch 1/10133442 not found, get inferred vector 

4024 not found, get inferred vector 
(200, 2, 1)
115033 not found, get inferred vector 
45709 not found, get inferred vector 
(200, 2, 1)
129707 not found, get inferred vector 
127155 not found, get inferred vector 
(200, 2, 1)
3540 not found, get inferred vector 
3219 not found, get inferred vector 
(200, 2, 1)
8949 not found, get inferred vector 
2919 not found, get inferred vector 
(200, 2, 1)
8863 not found, get inferred vector 
4341 not found, get inferred vector 
(200, 2, 1)
6149 not found, get inferred vector 
39850 not found, get inferred vector 
(200, 2, 1)
27843 not found, get inferred vector 
66950 not found, get inferred vector 
(200, 2, 1)
115138 not found, get inferred vector 
134515 not found, get inferred vector 
(200, 2, 1)
5280 not found, get inferred vector 
3139 not found, get inferred vector 
(200, 2, 1)
7995 not found, get inferred vector 
62277 not found, get inferred vector 
(200, 2, 1)
140418 not found, get infe

KeyError: '2893'

In [ ]:
import numpy
# vstack
from numpy import vstack
from numpy import hstack

a1 = np.zeros((1, 2,512))
# print(a1)
print(a1.shape)
a2 = np.ones((1, 2,512))
# print(a2)
print(a2.shape)
a3 = np.ones((1, 2,512))
# print(a3)
print(a3.shape)
a4 = vstack((a1, a2, a3))
print(a4)
print(a4.shape)


a1 = np.zeros((512,1))
# print(a1)
print(a1.shape)
a2 = np.ones((512,1))
# print(a2)
print(a2.shape)
a3 = np.ones((512,1))
# print(a3)
print(a3.shape)
a4 = hstack((a1, a2, a3))
print(a4)
print(a4.shape)

In [ ]:
import numpy
# vstack
from numpy import vstack
from numpy import hstack
from numpy import dstack

a1 = np.zeros((2, 1))
# print(a1)
print(a1.shape)
a2 = np.ones((2, 1))
# print(a2)
print(a2.shape)

a3 = hstack((a1, a2))
# print(a3)
print(a3.shape)

a4 = dstack((a3, a3, a3))
print(a4.shape)
print(a4)
